In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
import sys
sys.path.insert(0, '..//')
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path

from src.common import schema, enums, functions


In [80]:
path_results = Path(r"C:\Users\sceac10\OneDrive - Cardiff University\04 - Projects\22 - Heat demand scotland\data\Results_by_LAs")
path_full_results = Path(r"C:\Users\sceac10\OneDrive - Cardiff University\04 - Projects\22 - Heat demand scotland\data")

In [81]:
path_dwellings_2018 = Path(r"C:\Users\sceac10\OneDrive - Cardiff University\04 - Projects\22 - Heat demand scotland\data\census data and dwellings\hh-est-by-2011-dz-small-area-14-21.xlsx")

In [82]:
census_year = 2011
target_year = 2018
dwellings_2018 = pd.read_excel(path_dwellings_2018, sheet_name=f"{target_year}", header=3, index_col=0)
dwellings_2018 = dwellings_2018["Total number of dwellings"].to_frame()
dwellings_2018.columns = [f"Total number of dwellings {target_year}"]

In [83]:
dwellings_2018

,Total number of dwellings 2018
2011 Data Zone Code,
S01006506,492
S01006507,381
S01006508,326
S01006509,281
S01006510,321
...,...
S01013477,321
S01013478,362
S01013479,467


In [84]:
def concatenate_EPC_results()->pd.DataFrame:
  """Concatenate all EPC results"""
  frames = []
  for temp_filepath in path_results.glob("*.csv"):
    temp_dataf = pd.read_csv(temp_filepath, index_col=0)
    frames.append(temp_dataf)
  if len(frames)>0:
    full_epc_df = pd.concat(frames, axis=0)
  return full_epc_df

full_epc_df = concatenate_EPC_results()
full_epc_df.to_csv(path_full_results/"full_scotland_EPC_results.csv")

In [85]:
full_epc_df[schema.geoLookupSchema.ladnm].unique()

array(['aberdeen city', 'aberdeenshire', 'angus', 'dundee city',
       'argyll and bute', 'city of edinburgh', 'midlothian',
       'west lothian', 'clackmannanshire', 'fife',
       'dumfries and galloway', 'east ayrshire', 'east renfrewshire',
       'south ayrshire', 'east dunbartonshire', 'north lanarkshire',
       'east lothian', 'glasgow city', 'falkirk', 'stirling',
       'renfrewshire', 'south lanarkshire', 'na heileanan siar',
       'highland', 'inverclyde', 'scottish borders', 'moray',
       'north ayrshire', 'orkney islands', 'perth and kinross',
       'shetland islands', 'west dunbartonshire'], dtype=object)

In [86]:

census_data_lsoa_level = pd.read_csv(path_full_results/"formatted_census_data_lsoa_level.csv", index_col=0)
census_data_lsoa_level = census_data_lsoa_level.pivot(index=schema.finalCensusSchema.lsoa , columns=schema.finalCensusSchema.dwelling_category, values = schema.finalCensusSchema.number)
census_data_lsoa_level.columns = [f'Number of {c} in {census_year}' for c in census_data_lsoa_level.columns]
census_data_lsoa_level[schema.skeletonSchema.total_number_dwelling] = census_data_lsoa_level.sum(axis=1)
census_data_lsoa_level

,Number of detached gas boiler in 2011,Number of detached oil boiler in 2011,Number of detached resistance in 2011,Number of detached solid fuel boiler in 2011,Number of flat gas boiler in 2011,Number of flat oil boiler in 2011,Number of flat resistance in 2011,Number of flat solid fuel boiler in 2011,Number of semi-detached gas boiler in 2011,Number of semi-detached oil boiler in 2011,Number of semi-detached resistance in 2011,Number of semi-detached solid fuel boiler in 2011,Number of terraced gas boiler in 2011,Number of terraced oil boiler in 2011,Number of terraced resistance in 2011,Number of terraced solid fuel boiler in 2011,Total number of dwellings in 2011
lsoa11cd,,,,,,,,,,,,,,,,,
S01006506,85.0,10.0,0.0,1.0,158.0,0.0,63.0,1.0,50.0,6.0,2.0,0.0,30.0,0.0,1.0,1.0,408.0
S01006507,84.0,2.0,0.0,0.0,49.0,0.0,25.0,1.0,51.0,0.0,8.0,0.0,114.0,0.0,23.0,0.0,357.0
S01006508,43.0,0.0,1.0,0.0,168.0,0.0,59.0,0.0,18.0,0.0,4.0,0.0,36.0,0.0,2.0,0.0,331.0
S01006509,25.0,0.0,4.0,1.0,53.0,0.0,29.0,0.0,64.0,0.0,9.0,1.0,68.0,0.0,5.0,1.0,260.0
S01006510,32.0,2.0,0.0,0.0,67.0,1.0,8.0,0.0,96.0,2.0,2.0,0.0,35.0,1.0,1.0,0.0,247.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S01013477,22.0,0.0,0.0,0.0,133.0,0.0,23.0,1.0,41.0,0.0,2.0,0.0,67.0,0.0,4.0,0.0,293.0
S01013478,86.0,2.0,1.0,0.0,112.0,0.0,5.0,1.0,60.0,0.0,3.0,0.0,75.0,0.0,5.0,0.0,350.0
S01013479,53.0,1.0,1.0,0.0,177.0,5.0,49.0,0.0,44.0,0.0,5.0,0.0,71.0,0.0,6.0,1.0,413.0


In [87]:
full_epc_df = full_epc_df.merge(census_data_lsoa_level, left_index=True, right_index=True, how="left")


In [88]:
def linear_extrapolation_of_census_data(target_number_dwellings:pd.DataFrame, original_census_data:pd.DataFrame, target_year:int)->pd.DataFrame:
  """Linear projection of the census data"""
  extrapolated_census_data = pd.merge(original_census_data, target_number_dwellings, right_index=True, left_index=True)
  factors = extrapolated_census_data[f"Total number of dwellings {target_year}"]/census_data_lsoa_level[schema.skeletonSchema.total_number_dwelling]
  extrapolated_census_data = extrapolated_census_data.mul(factors, axis=0).round(0)
  new_cols_name = [x.replace(f"{census_year}", f"{target_year}") for x in extrapolated_census_data.columns]
  extrapolated_census_data.columns = new_cols_name
  return extrapolated_census_data.iloc[:, :-2]


In [89]:
census_data_extrpolated = linear_extrapolation_of_census_data(dwellings_2018, census_data_lsoa_level, target_year)
census_data_extrpolated.head()

,Number of detached gas boiler in 2018,Number of detached oil boiler in 2018,Number of detached resistance in 2018,Number of detached solid fuel boiler in 2018,Number of flat gas boiler in 2018,Number of flat oil boiler in 2018,Number of flat resistance in 2018,Number of flat solid fuel boiler in 2018,Number of semi-detached gas boiler in 2018,Number of semi-detached oil boiler in 2018,Number of semi-detached resistance in 2018,Number of semi-detached solid fuel boiler in 2018,Number of terraced gas boiler in 2018,Number of terraced oil boiler in 2018,Number of terraced resistance in 2018,Number of terraced solid fuel boiler in 2018
S01006506,102.0,12.0,0.0,1.0,191.0,0.0,76.0,1.0,60.0,7.0,2.0,0.0,36.0,0.0,1.0,1.0
S01006507,90.0,2.0,0.0,0.0,52.0,0.0,27.0,1.0,54.0,0.0,9.0,0.0,122.0,0.0,25.0,0.0
S01006508,42.0,0.0,1.0,0.0,165.0,0.0,58.0,0.0,18.0,0.0,4.0,0.0,35.0,0.0,2.0,0.0
S01006509,27.0,0.0,4.0,1.0,57.0,0.0,31.0,0.0,69.0,0.0,10.0,1.0,73.0,0.0,5.0,1.0
S01006510,42.0,3.0,0.0,0.0,87.0,1.0,10.0,0.0,125.0,3.0,3.0,0.0,45.0,1.0,1.0,0.0


In [90]:
full_epc_df = full_epc_df.merge(census_data_extrpolated, left_index=True, right_index=True, how="left")

In [91]:
# Remove uncategorized dwellings
columns_to_remove = [x for x in full_epc_df.columns if enums.heating_systems.UNCATEGORIZED.name.lower() in x]
print(columns_to_remove)
full_epc_df = full_epc_df.drop(columns_to_remove, axis=1)

['Average heat demand before energy efficiency measures for detached uncategorized (kWh)', 'Average heat demand before energy efficiency measures for flat uncategorized (kWh)', 'Average heat demand before energy efficiency measures for semi-detached uncategorized (kWh)', 'Average heat demand before energy efficiency measures for terraced uncategorized (kWh)', 'Average heat demand after energy efficiency measures for detached uncategorized (kWh)', 'Average heat demand after energy efficiency measures for flat uncategorized (kWh)', 'Average heat demand after energy efficiency measures for semi-detached uncategorized (kWh)', 'Average heat demand after energy efficiency measures for terraced uncategorized (kWh)', 'Average floor area of detached uncategorized (m2)', 'Average floor area of flat uncategorized (m2)', 'Average floor area of semi-detached uncategorized (m2)', 'Average floor area of terraced uncategorized (m2)']


In [92]:
dwel_categories = functions.get_list_dwelling_categories()

for temp_year in [census_year, target_year]:
  before_total_cols = []
  after_total_cols = []
  for dwel_cat in dwel_categories:
    before_heat_demand_col = f"Average heat demand before energy efficiency measures for {dwel_cat} (kWh)"
    after_heat_demand_col = f"Average heat demand after energy efficiency measures for {dwel_cat} (kWh)"
    number_col = f'Number of {dwel_cat} in {temp_year}'
    before_total_heat_col = f"Total heat demand before energy efficiency measures of {dwel_cat} in {temp_year} (kWh)"
    after_total_heat_col = f"Total heat demand after energy efficiency measures of {dwel_cat} in {temp_year} (kWh)"
    if number_col in full_epc_df.columns:
      full_epc_df[before_total_heat_col] = full_epc_df[before_heat_demand_col]*full_epc_df[number_col]
      full_epc_df[after_total_heat_col] = full_epc_df[after_heat_demand_col]*full_epc_df[number_col]
      before_total_cols.append(before_total_heat_col)
      after_total_cols.append(after_total_heat_col)

    full_epc_df[f"Total heat demand before energy efficiency measures {temp_year} (kWh)"] = full_epc_df[before_total_cols].sum(axis=1)
    full_epc_df[f"Total heat demand after energy efficiency measures {temp_year} (kWh)"] = full_epc_df[after_total_cols].sum(axis=1)

In [93]:
full_epc_df.columns

Index(['msoa11cd', 'ladcd', 'ladnm',
       'Average heat demand before energy efficiency measures for detached district heating (kWh)',
       'Average heat demand before energy efficiency measures for detached gas boiler (kWh)',
       'Average heat demand before energy efficiency measures for detached heat pump (kWh)',
       'Average heat demand before energy efficiency measures for detached oil boiler (kWh)',
       'Average heat demand before energy efficiency measures for detached resistance (kWh)',
       'Average heat demand before energy efficiency measures for detached solid fuel boiler (kWh)',
       'Average heat demand before energy efficiency measures for flat district heating (kWh)',
       ...
       'Total heat demand before energy efficiency measures of detached oil boiler in 2018 (kWh)',
       'Total heat demand after energy efficiency measures of detached oil boiler in 2018 (kWh)',
       'Total heat demand before energy efficiency measures of flat resistance in 2

In [94]:
full_epc_df.index.name = schema.skeletonSchema.lsoa
full_epc_df.head()

,msoa11cd,ladcd,ladnm,Average heat demand before energy efficiency measures for detached district heating (kWh),Average heat demand before energy efficiency measures for detached gas boiler (kWh),Average heat demand before energy efficiency measures for detached heat pump (kWh),Average heat demand before energy efficiency measures for detached oil boiler (kWh),Average heat demand before energy efficiency measures for detached resistance (kWh),Average heat demand before energy efficiency measures for detached solid fuel boiler (kWh),Average heat demand before energy efficiency measures for flat district heating (kWh),...,Total heat demand before energy efficiency measures of detached oil boiler in 2018 (kWh),Total heat demand after energy efficiency measures of detached oil boiler in 2018 (kWh),Total heat demand before energy efficiency measures of flat resistance in 2018 (kWh),Total heat demand after energy efficiency measures of flat resistance in 2018 (kWh),Total heat demand before energy efficiency measures of flat gas boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat gas boiler in 2018 (kWh),Total heat demand before energy efficiency measures of flat solid fuel boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat solid fuel boiler in 2018 (kWh),Total heat demand before energy efficiency measures of flat oil boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat oil boiler in 2018 (kWh)
lsoa11cd,,,,,,,,,,,,,,,,,,,,,
S01006506,S02001236,S12000033,aberdeen city,19887.0,21438.468750,16779.750000,19968.000000,14088.5,31384.75,3178.44,...,239616.000000,230260.060738,524564.090909,357768.209080,1.498661e+06,1.212025e+06,26119.0,14889.283257,0.000000,0.000000
S01006507,S02001236,S12000033,aberdeen city,19887.0,21655.277778,24595.733333,21845.000000,10184.0,31384.75,3178.44,...,43690.000000,27515.119837,253269.000000,151087.556211,4.951940e+05,3.861348e+05,26119.0,14889.283257,0.000000,0.000000
S01006508,S02001236,S12000033,aberdeen city,19887.0,22849.600000,24595.733333,27226.631579,13177.0,31384.75,3178.44,...,0.000000,0.000000,405157.388889,271352.684156,1.318155e+06,1.118214e+06,0.0,0.000000,0.000000,0.000000
S01006509,S02001236,S12000033,aberdeen city,19887.0,20255.000000,12288.000000,27226.631579,12411.0,31384.75,3178.44,...,0.000000,0.000000,241525.769231,143893.714008,4.730610e+05,3.747631e+05,0.0,0.000000,0.000000,0.000000
S01006510,S02001236,S12000033,aberdeen city,19887.0,22805.769231,24595.733333,27226.631579,14088.5,31384.75,3178.44,...,81679.894737,61444.073866,57800.000000,23745.007514,6.947189e+05,5.595759e+05,0.0,0.000000,15683.666667,9060.830466


In [95]:
full_epc_df = functions.add_road_length(full_epc_df.reset_index())
full_epc_df = functions.add_area_LSOA(full_epc_df)
full_epc_df

,lsoa11cd,msoa11cd,ladcd,ladnm,Average heat demand before energy efficiency measures for detached district heating (kWh),Average heat demand before energy efficiency measures for detached gas boiler (kWh),Average heat demand before energy efficiency measures for detached heat pump (kWh),Average heat demand before energy efficiency measures for detached oil boiler (kWh),Average heat demand before energy efficiency measures for detached resistance (kWh),Average heat demand before energy efficiency measures for detached solid fuel boiler (kWh),...,Total heat demand before energy efficiency measures of flat resistance in 2018 (kWh),Total heat demand after energy efficiency measures of flat resistance in 2018 (kWh),Total heat demand before energy efficiency measures of flat gas boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat gas boiler in 2018 (kWh),Total heat demand before energy efficiency measures of flat solid fuel boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat solid fuel boiler in 2018 (kWh),Total heat demand before energy efficiency measures of flat oil boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat oil boiler in 2018 (kWh),Road length (m),Area (km2)
0,S01006506,S02001236,S12000033,aberdeen city,19887.0,21438.468750,16779.750000,19968.000000,14088.500000,31384.750000,...,524564.090909,357768.209080,1.498661e+06,1.212025e+06,26119.0,14889.283257,0.000000,0.000000,7144.704905,4.388802
1,S01006507,S02001236,S12000033,aberdeen city,19887.0,21655.277778,24595.733333,21845.000000,10184.000000,31384.750000,...,253269.000000,151087.556211,4.951940e+05,3.861348e+05,26119.0,14889.283257,0.000000,0.000000,3562.459752,0.221747
2,S01006508,S02001236,S12000033,aberdeen city,19887.0,22849.600000,24595.733333,27226.631579,13177.000000,31384.750000,...,405157.388889,271352.684156,1.318155e+06,1.118214e+06,0.0,0.000000,0.000000,0.000000,2689.130582,0.270195
3,S01006509,S02001236,S12000033,aberdeen city,19887.0,20255.000000,12288.000000,27226.631579,12411.000000,31384.750000,...,241525.769231,143893.714008,4.730610e+05,3.747631e+05,0.0,0.000000,0.000000,0.000000,2029.960587,0.096254
4,S01006510,S02001236,S12000033,aberdeen city,19887.0,22805.769231,24595.733333,27226.631579,14088.500000,31384.750000,...,57800.000000,23745.007514,6.947189e+05,5.595759e+05,0.0,0.000000,15683.666667,9060.830466,2312.863906,0.180077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7020,S01013477,S02002513,S12000040,west lothian,48390.0,16519.909091,14548.333333,28218.469565,16557.766234,11968.000000,...,225945.000000,82078.717467,1.221030e+06,9.465265e+05,9964.0,6832.617762,0.000000,0.000000,1736.366761,0.109882
7021,S01013478,S02002514,S12000040,west lothian,48390.0,13024.666667,35217.000000,25249.000000,19586.000000,26728.933333,...,29540.000000,10822.619469,8.489240e+05,7.284484e+05,9964.0,6832.617762,0.000000,0.000000,2139.181346,0.124382
7022,S01013479,S02002514,S12000040,west lothian,48390.0,15157.238095,35217.000000,25249.000000,19586.000000,26728.933333,...,446365.000000,250825.525105,1.596003e+06,1.296303e+06,0.0,0.000000,85239.818182,46102.274206,2648.023492,0.267146
7023,S01013480,S02002514,S12000040,west lothian,48390.0,10854.022727,35217.000000,25249.000000,19586.000000,26728.933333,...,7849.769231,4308.880519,1.443360e+05,1.171760e+05,0.0,0.000000,0.000000,0.000000,1720.318816,0.096247


In [96]:
full_epc_df.to_csv(path_full_results/"full_scotland_results.csv")

In [97]:
full_epc_df.head()

,lsoa11cd,msoa11cd,ladcd,ladnm,Average heat demand before energy efficiency measures for detached district heating (kWh),Average heat demand before energy efficiency measures for detached gas boiler (kWh),Average heat demand before energy efficiency measures for detached heat pump (kWh),Average heat demand before energy efficiency measures for detached oil boiler (kWh),Average heat demand before energy efficiency measures for detached resistance (kWh),Average heat demand before energy efficiency measures for detached solid fuel boiler (kWh),...,Total heat demand before energy efficiency measures of flat resistance in 2018 (kWh),Total heat demand after energy efficiency measures of flat resistance in 2018 (kWh),Total heat demand before energy efficiency measures of flat gas boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat gas boiler in 2018 (kWh),Total heat demand before energy efficiency measures of flat solid fuel boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat solid fuel boiler in 2018 (kWh),Total heat demand before energy efficiency measures of flat oil boiler in 2018 (kWh),Total heat demand after energy efficiency measures of flat oil boiler in 2018 (kWh),Road length (m),Area (km2)
0,S01006506,S02001236,S12000033,aberdeen city,19887.0,21438.468750,16779.750000,19968.000000,14088.5,31384.75,...,524564.090909,357768.209080,1.498661e+06,1.212025e+06,26119.0,14889.283257,0.000000,0.000000,7144.704905,4.388802
1,S01006507,S02001236,S12000033,aberdeen city,19887.0,21655.277778,24595.733333,21845.000000,10184.0,31384.75,...,253269.000000,151087.556211,4.951940e+05,3.861348e+05,26119.0,14889.283257,0.000000,0.000000,3562.459752,0.221747
2,S01006508,S02001236,S12000033,aberdeen city,19887.0,22849.600000,24595.733333,27226.631579,13177.0,31384.75,...,405157.388889,271352.684156,1.318155e+06,1.118214e+06,0.0,0.000000,0.000000,0.000000,2689.130582,0.270195
3,S01006509,S02001236,S12000033,aberdeen city,19887.0,20255.000000,12288.000000,27226.631579,12411.0,31384.75,...,241525.769231,143893.714008,4.730610e+05,3.747631e+05,0.0,0.000000,0.000000,0.000000,2029.960587,0.096254
4,S01006510,S02001236,S12000033,aberdeen city,19887.0,22805.769231,24595.733333,27226.631579,14088.5,31384.75,...,57800.000000,23745.007514,6.947189e+05,5.595759e+05,0.0,0.000000,15683.666667,9060.830466,2312.863906,0.180077
